 # projeto sem objetivo pre definido

* primeiro fazer coleta dos dados para armazenamento 
*  sera feito coleta atraves de web scraping
* site: https://www.zapimoveis.com.br
* limpeza de dados
* talvez crie um banco de dados para mostrar a criacao de um banco de dados
* talvez pode ser feito uma coleta diaria durante um tempo para saber qto tempo um imovel fica disponivel para venda, qual o tempo medio de espera para ser vendido, qual o preço medio dos  imoveis mais vendidos 

apos coleta sera feita uma analise exploratoria para saber o que sera possivel de analisar, como por exemplo, valor media das casa, valor medio por regiao, a media de comodos, regioes  e comodos afetam os preços, qual tem mais impacto...?

tecnologia esperadas q serao usadas

* python
    * selenium ou beutiful soup para coleta de dados
    * pandas para exploraçao e limpeza

* power bi
    * para apresentaçao final

*github para fazer os historicos de evoluçao diaria e ter noçao de qto tempo levou para fazer o projeto (nao estou me obrigando a ter prazo, tenho outro trabalho entao sera feito nas horas disponiveis sem afetar outros compromissos)






In [1]:
#importando as bibliotecas necessarias
import pandas as pd
import selenium 
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from datetime import date

In [2]:
# explarando o site com filtro de casa para venda e na ordem de maiores valores
#driver_casa = webdriver.Chrome()
#driver_casa.get('https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina=1&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda')
#driver_casa.maximize_window()
#original_window = driver_casa.current_window_handle

In [3]:
#pegando os anuncios das casas
#anuncios = driver_casa.find_elements(By.CLASS_NAME, 'card-container')

In [4]:
#conferindo a quantidade de elementos 
#len(anuncios)

entrando no anuncio para verificar o que é possivel extrair 

fazendo uma breve verificacao em algums anuncio vi q os valores nao correspondem ao valor real,
 entao apos a coleta de dados sera necessario ainda uma limpeza 

 aqui ja da para extrair :

 valor do imovel
 
 condominio
 
 iptu
 
 area
 
 quantidade de banheiro
 
 quantidade de quartos
 
 quantidade de vagas na garagem
 
 rua, numero e bairro (cidade e estado sao sempre iguais, ja q é uma pesquisa de imoveis de sao jose dos campos)
 
 se esta por imobiliaria ou nao
 
 extrair tambem as informaçoes das descriçoes para complentar alguma informaçao
 
 a descricao tbm pode estar como caracteristica
 
 quantidade de andares
    

In [5]:
# window_handles eh uma lista de todas as guias q estao abertas, usar esse for pra saber quais os nomes das guias
#for window_handle in driver_casa.window_handles:
 #   print(window_handle)

In [6]:
#original_driver = driver_casa.current_window_handle
#original_driver

In [7]:
#url = driver_casa.current_url
#url = url.replace('pagina=1', 'pagina=2')
#print(url)

In [8]:
'''
#dados = pd.DataFrame(columns= ['Endereço', 'Preço', 'Condominio', 'Iptu', 'Lista Comodos','Lista Caracteristicas', 'Obserevacoes'])

for indice in range(121,161):
    driver_casa = webdriver.Chrome()
    url = f'https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina={indice}&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda' 
    print(f'indice= {indice}')
    print()
    print(f'url = {url}')
    driver_casa.get(url)
    driver_casa.maximize_window()
    original_window = driver_casa.current_window_handle
    print(f'original_window == {original_window}')
    
    anuncios = driver_casa.find_elements(By.CLASS_NAME, 'card-container')

    for i in range(len(anuncios)):
        try:
            while len(driver_casa.window_handles) == 1: # fazer um break de emergencia
                print()
                print()
                print(f'indice == {i}')
                print(f'tamanho de len de driver_casa.window_handles == {len(driver_casa.window_handles)}')
                print(f'current window == {driver_casa.current_window_handle}')

                try:
                    print(driver_casa.current_window_handle)
                    print(f'indice == {i}')
                    driver_casa.find_elements(By.CLASS_NAME, 'card-container')[i].click()
                    print(len(driver_casa.window_handles))
                except:
                    print('erro no click do anuncio')
                if len(driver_casa.window_handles) > 1 :
                    driver_casa.switch_to.window(driver_casa.window_handles[1])
                    break


            try:
                endereco= driver_casa.find_element(By.CLASS_NAME, 'info__map-link ') 
                print(f'Endereço = {endereco.text}')
            except:
                print('erro endereco =====================')

            try:
                preco = driver_casa.find_element(By.CLASS_NAME, 'price__item--main')
                print(f'preco = {preco.text}')
            except:
                print('erro preco =====================')

            try:
                condominio = driver_casa.find_elements(By.CLASS_NAME, 'price__item')[0]
                print(f'condominio = {condominio.text}')
            except:
                print('erro condominio =====================')

            try:
                iptu = driver_casa.find_elements(By.CLASS_NAME, 'price__item')[1]
                print(f'iptu = {iptu.text}')
            except:
                print('erro iptu =====================')

            try:
                comodos = driver_casa.find_elements(By.CLASS_NAME, 'feature__item')
                lista_comodos = [comodo.text for comodo in comodos]
                print(f'lista de comodos = {lista_comodos}')
            except:
                print('erro comodos =====================')

            try:
                caracteristicas = driver_casa.find_elements(By.CLASS_NAME, 'amenities__list-item')
                lista_caracteristicas = [caracteristica.text for caracteristica in caracteristicas]
                print(f'Caracteristica = {lista_caracteristicas}')
            except:
                print('erro caracteristicas =====================')

            try:
                obserevacoes = driver_casa.find_element(By.CLASS_NAME, 'amenities__base-description')
                print(f'obserevacoes = {obserevacoes.text}')
            except:
                print('erro obserevacoes =====================')

            try:
                url_id_anuncio = driver_casa.current_url
                print(f'url_id_anuncio = {url_id_anuncio}')
            except:
                print('erro url_id_anuncio =====================')
            

            #print()
            #print()
            #print(f'indice ===={i}')
            #print(driver_casa.current_window_handle)
                #print()
            #print(f'Endereço = {endereco.text}')
            #print(f'preco = {preco.text}')
            #print(f'condominio = {condominio.text}')
            #print(f'iptu = {iptu.text}')
            #print(f'Area = {lista_comodos[4]}')
            #print(f'Quartos = {lista_comodos[5]}')
            #print(f'Banheiros = {lista_comodos[-1]}')
            #print(f'Caracteristica = {caracteristica}')
            #print(f'obserevacoes = {obserevacoes.text}')
            #print()
            #print()


            caracteristica = [caracteristica for caracteristica in caracteristicas]
            dic_dados = {'Endereço':[endereco.text],
                     'Preço' : [preco.text],
                     'Condominio': [condominio.text],
                     'Iptu': [iptu.text],
                     'Lista Comodos': lista_comodos,
                     'Lista Caracteristicas' : lista_caracteristicas,
                     'Obserevacoes' : [obserevacoes.text],
                     'url_id_anuncio' : [url_id_anuncio]}


            while len(driver_casa.window_handles) > 1:
                driver_casa.switch_to.window(driver_casa.window_handles[1])
                print(driver_casa.current_window_handle)
                driver_casa.close()
                print(f'len(driver_casa.window_handles) ==== {len(driver_casa.window_handles)}')
                driver_casa.switch_to.window(original_window)
                print(driver_casa.current_window_handle)

            print()
            print()
            #print(f'indice ========== {i}')
            #print(dic_dados)
            #print()
            #print()
            dados = dados.append(dic_dados, ignore_index = True)
        except:
            print()
            print()
            print('************ FALHA de CARREGAR anuncio!!!!!!!!!!!!!!!!')
            print()
            print()
    driver_casa.quit()
dados'''

"\n#dados = pd.DataFrame(columns= ['Endereço', 'Preço', 'Condominio', 'Iptu', 'Lista Comodos','Lista Caracteristicas', 'Obserevacoes'])\n\nfor indice in range(121,161):\n    driver_casa = webdriver.Chrome()\n    url = f'https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina={indice}&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda' \n    print(f'indice= {indice}')\n    print()\n    print(f'url = {url}')\n    driver_casa.get(url)\n    driver_casa.maximize_window()\n    original_window = driver_casa.current_window_handle\n    print(f'original_window == {original_window}')\n    \n    anuncios = driver_casa.find_elements(By.CLASS_NAME, 'card-container')\n\n    for i in range(len(anuncios)):\n        try:\n            while len(driver_casa.window_handles) == 1: # fazer um break de emergencia\n        

In [9]:
#dados2

In [10]:
#dados

In [11]:
#data_atual = date.today()
#data_atual = f'{data_atual.day}_{data_atual.month}_{data_atual.year}'
#tabela_imoveis_raw = dados.to_csv(f'tabela_imoveis_raw_{data_atual}.csv', sep = ',', index = False)

In [12]:
#df = pd.read_csv('tabela_imoveis_raw_12_3_2022.csv')
#df

In [13]:
#data_atual = date.today()
#data_atual = f'{data_atual.day}_{data_atual.month}_{data_atual.year}'
#data_atual

In [ ]:
#dados2 = pd.DataFrame(columns= ['Endereço', 'Preço', 'Condominio', 'Iptu', 'Lista Comodos','Lista Caracteristicas', 'Obserevacoes'])

for indice in range(301,351):
    driver_casa = webdriver.Chrome()
    url = f'https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina={indice}&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda' 
    print(f'indice= {indice}')
    print()
    print(f'url = {url}')
    driver_casa.get(url)
    driver_casa.maximize_window()
    original_window = driver_casa.current_window_handle
    print(f'original_window == {original_window}')
    
    anuncios = driver_casa.find_elements(By.CLASS_NAME, 'card-container')

    for i in range(len(anuncios)):
        try:
            while len(driver_casa.window_handles) == 1: # fazer um break de emergencia
                print()
                print()
                print(f'indice == {i}')
                print(f'tamanho de len de driver_casa.window_handles == {len(driver_casa.window_handles)}')
                print(f'current window == {driver_casa.current_window_handle}')

                try:
                    print(driver_casa.current_window_handle)
                    print(f'indice == {i}')
                    driver_casa.find_elements(By.CLASS_NAME, 'card-container')[i].click()
                    print(len(driver_casa.window_handles))
                except:
                    print('erro no click do anuncio')
                if len(driver_casa.window_handles) > 1 :
                    driver_casa.switch_to.window(driver_casa.window_handles[1])
                    break


            try:
                endereco= driver_casa.find_element(By.CLASS_NAME, 'info__map-link ') 
                print(f'Endereço = {endereco.text}')
            except:
                print('erro endereco =====================')

            try:
                preco = driver_casa.find_element(By.CLASS_NAME, 'price__item--main')
                print(f'preco = {preco.text}')
            except:
                print('erro preco =====================')

            try:
                condominio = driver_casa.find_elements(By.CLASS_NAME, 'price__item')[0]
                print(f'condominio = {condominio.text}')
            except:
                print('erro condominio =====================')

            try:
                iptu = driver_casa.find_elements(By.CLASS_NAME, 'price__item')[1]
                print(f'iptu = {iptu.text}')
            except:
                print('erro iptu =====================')

            try:
                comodos = driver_casa.find_elements(By.CLASS_NAME, 'feature__item')
                lista_comodos = [comodo.text for comodo in comodos]
                print(f'lista de comodos = {lista_comodos}')
            except:
                print('erro comodos =====================')

            try:
                caracteristicas = driver_casa.find_elements(By.CLASS_NAME, 'amenities__list-item')
                lista_caracteristicas = [caracteristica.text for caracteristica in caracteristicas]
                print(f'Caracteristica = {lista_caracteristicas}')
            except:
                print('erro caracteristicas =====================')

            try:
                obserevacoes = driver_casa.find_element(By.CLASS_NAME, 'amenities__base-description')
                print(f'obserevacoes = {obserevacoes.text}')
            except:
                print('erro obserevacoes =====================')

            try:
                url_id_anuncio = driver_casa.current_url
                print(f'url_id_anuncio = {url_id_anuncio}')
            except:
                print('erro url_id_anuncio =====================')
            

            #print()
            #print()
            #print(f'indice ===={i}')
            #print(driver_casa.current_window_handle)
                #print()
            #print(f'Endereço = {endereco.text}')
            #print(f'preco = {preco.text}')
            #print(f'condominio = {condominio.text}')
            #print(f'iptu = {iptu.text}')
            #print(f'Area = {lista_comodos[4]}')
            #print(f'Quartos = {lista_comodos[5]}')
            #print(f'Banheiros = {lista_comodos[-1]}')
            #print(f'Caracteristica = {caracteristica}')
            #print(f'obserevacoes = {obserevacoes.text}')
            #print()
            #print()


            caracteristica = [caracteristica for caracteristica in caracteristicas]
            dic_dados = {'Endereço':endereco.text,
                     'Preço' : preco.text,
                     'Condominio': condominio.text,
                     'Iptu': iptu.text,
                     'Lista Comodos': lista_comodos,
                     'Lista Caracteristicas' : lista_caracteristicas,
                     'Obserevacoes' : obserevacoes.text,
                     'url_id_anuncio' : url_id_anuncio}


            while len(driver_casa.window_handles) > 1:
                driver_casa.switch_to.window(driver_casa.window_handles[1])
                print(driver_casa.current_window_handle)
                driver_casa.close()
                print(f'len(driver_casa.window_handles) ==== {len(driver_casa.window_handles)}')
                driver_casa.switch_to.window(original_window)
                print(driver_casa.current_window_handle)

            print()
            print()
            #print(f'indice ========== {i}')
            #print(dic_dados)
            #print()
            #print()
            dados2 = dados2.append(dic_dados, ignore_index = True)
        except:
            print()
            print()
            print('************ FALHA de CARREGAR anuncio!!!!!!!!!!!!!!!!')
            print()
            print()
    driver_casa.quit()
dados2

indice= 301

url = https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina=301&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda
original_window == CDwindow-64D52D7B68C35F2CDC36EB2212782E98


indice == 0
tamanho de len de driver_casa.window_handles == 1
current window == CDwindow-64D52D7B68C35F2CDC36EB2212782E98
CDwindow-64D52D7B68C35F2CDC36EB2212782E98
indice == 0
2
Endereço = Rua Marcelo Cordeiro de Oliveira, 415 - Residencial de Ville, São José dos Campos - SP
preco = R$ 426.000
condominio = condomínio R$ 269
iptu = IPTU R$ 500
lista de comodos = ['', '', '', '', '85 m²', '2 quartos', '1 vaga', '2 banheiros']
Caracteristica = ['Academia', 'Quintal', 'Varanda']
obserevacoes = Sobrado à Venda, 85m² por R$ 426,000. Residencial de Ville - São José dos Campos/SP

Encontrar o imóvel dos sonhos nunca f

C:\Users\jpnun\AppData\Local\Temp\ipykernel_10992\543639192.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados2 = dados2.append(dic_dados, ignore_index = True)


2
Endereço = Rua Bertolino Cezário dos Santos - Bosque dos Eucaliptos, São José dos Campos - SP
preco = R$ 426.000
condominio = condomínio R$ 600
iptu = IPTU R$ 41
lista de comodos = ['', '', '', '', '110 m²', '3 quartos', '1 vaga', '2 banheiros']
Caracteristica = ['Aceita animais', 'Armário embutido', 'Armário embutido no quarto', 'Armário na cozinha', 'Armário no banheiro', '', 'Conexão à internet', 'Piscina', 'Portaria 24h', 'Quadra poliesportiva', 'Salão de festas']
obserevacoes = CRECI-181266

Casa Condomínio - Bosque dos Eucaliptos - Condomínio Vilage Amsterdã - 3 dormitórios - 110m².

Esta casa está localizada no Condomínio Vilage Amsterdã no Bosque dos Eucaliptos, possui fácil acesso às principais vias da cidade.

Conheça um pouco mais das características desta linda casa:

- 110m²
- 3 dormitórios sendo uma suíte
- 1 vaga de garagem coberta
- Planejados em todos dormitórios e cozinha

Lazer do Condomínio:

- Piscina
- Quadra Poliesportiva
- Salão de Festas
- Playground
- Quadra

C:\Users\jpnun\AppData\Local\Temp\ipykernel_10992\543639192.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados2 = dados2.append(dic_dados, ignore_index = True)


2
Endereço = Rua Jurema Vinhas Betti - Residencial Dom Bosco, São José dos Campos - SP
preco = R$ 426.000
condominio = condomínio não informado
iptu = IPTU não informado
lista de comodos = ['', '', '', '', '150 m²', '3 quartos', '2 vagas', '2 banheiros']
Caracteristica = []
obserevacoes = Casa em ótimo estado
3 quartos com piso laminado
Uma suíte
Área de serviço
Garagem coberta para 2 carros

Agende a sua visita com um de nossos corretores
url_id_anuncio = https://www.zapimoveis.com.br/imovel/venda-casa-3-quartos-residencial-dom-bosco-sao-jose-dos-campos-sp-150m2-id-2537750231/
CDwindow-A0175C9EAFD6651960BA5004C472A942
len(driver_casa.window_handles) ==== 1
CDwindow-64D52D7B68C35F2CDC36EB2212782E98




indice == 3
tamanho de len de driver_casa.window_handles == 1
current window == CDwindow-64D52D7B68C35F2CDC36EB2212782E98
CDwindow-64D52D7B68C35F2CDC36EB2212782E98
indice == 3


...

1


indice == 20
tamanho de len de driver_casa.window_handles == 1
current window == CDwindow-263DBB87583000E0FBB8FCE645785B8E
CDwindow-263DBB87583000E0FBB8FCE645785B8E
indice == 20
2
Endereço = Urbanova, São José dos Campos - SP
preco = R$ 450.000
condominio = condomínio não informado
iptu = IPTU não informado
lista de comodos = ['', '', '', '', '142 m²', '3 quartos', '2 vagas', '2 banheiros']
Caracteristica = []
obserevacoes = Grande oportunidade 9.9270.4903
Entrada facilitada
Utilize seu FGTS
Imóvel acabamento fino, 3 dormitórios, 1 suite, 2 banheiros, quintal com churrasqueira, bem arejada, 2 vagas de garagem , próximo de mercados, farmácias, lojas, ponto de ônibus
Faça sua simulação
Marcelo Salu 9.9270.4903
Mostrar mais
url_id_anuncio = https://www.zapimoveis.com.br/imovel/venda-casa-3-quartos-urbanova-sao-jose-dos-campos-sp-142m2-id-2553430110/
CDwindow-35D915B25D3C5D8A3C4186E1746ACD12
len(driver_casa.window_handles) ==== 1
CDwindow-263DBB87583000E0FBB8FCE645785B8E




indice == 

C:\Users\jpnun\AppData\Local\Temp\ipykernel_10992\543639192.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados2 = dados2.append(dic_dados, ignore_index = True)


1


indice == 21
tamanho de len de driver_casa.window_handles == 1
current window == CDwindow-263DBB87583000E0FBB8FCE645785B8E
CDwindow-263DBB87583000E0FBB8FCE645785B8E
indice == 21
2
Endereço = Rua Cali - Jardim América, São José dos Campos - SP
preco = R$ 450.000
condominio = condomínio não informado
iptu = IPTU não informado
lista de comodos = ['', '', '', '', '88 m²', '2 quartos', '2 vagas', '2 banheiros']
Caracteristica = ['Edícula']
obserevacoes = Casa Residencial / Comercial
Jardim América

- Imóvel com 02 dormitórios amplos
- 02 Banheiros, um no interior do imóvel e um nos fundos
- Cozinha
- 02 vagas de garagem, sendo uma coberta.
- Edícula nos fundos, com opção de reverter para mais um dormitório.
- 150m² Área terreno
- 88m² Área construída

Contato: Sárgio – Imobiliária Grupo Kaza
WhatsApp:
CRECI: 163.548
Mostrar mais
url_id_anuncio = https://www.zapimoveis.com.br/imovel/venda-casa-2-quartos-com-edicula-jardim-america-sao-jose-dos-campos-sp-88m2-id-2552390690/
CDwindow-2BECFA

C:\Users\jpnun\AppData\Local\Temp\ipykernel_10992\543639192.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados2 = dados2.append(dic_dados, ignore_index = True)


1


indice == 22
tamanho de len de driver_casa.window_handles == 1
current window == CDwindow-263DBB87583000E0FBB8FCE645785B8E
CDwindow-263DBB87583000E0FBB8FCE645785B8E
indice == 22
2
Endereço = Rua Altair - Jardim Satélite, São José dos Campos - SP


In [26]:
dados2

,Endereço,Preço,Condominio,Iptu,Lista Comodos,Lista Caracteristicas,Obserevacoes,url_id_anuncio
0,"Jardim Vale do Sol, São José dos Campos - SP",R$ 390.000,condomínio não informado,IPTU não informado,"[, , , , 99 m², 3 quartos, 2 vagas, 2 banheiros]",[],A casa está localizado no bairro Jardim Vale d...,https://www.zapimoveis.com.br/imovel/venda-cas...
1,Rua Toshiyassu Watanabe - Jardim dos Bandeiran...,R$ 390.000,condomínio não informado,IPTU R$ 30,"[, , , , 160 m², 3 quartos, 3 vagas, 3 banheiros]","[Área de serviço, Churrasqueira, Cozinha, Lava...","Excelente Casa térrea em rua Publica ( Locado,...",https://www.zapimoveis.com.br/imovel/venda-cas...
2,"Rua Brumado, 142 - Jardim Vale do Sol, São Jos...",R$ 390.000,condomínio não informado,IPTU não informado,"[, , , , 131 m², 2 quartos, 2 vagas, 3 banheiros]",[],Casa Zona Sul de SJC!\n3 Dormitórios\n3 Banhei...,https://www.zapimoveis.com.br/imovel/venda-cas...
3,"Rua Santarem, 90 - Parque Industrial, São José...",R$ 390.000,condomínio não informado,IPTU não informado,"[, , , , 110 m², 3 quartos, 3 vagas, 2 banheiros]",[],Casa - Parque Industrial - 3 Dormitórios - 110...,https://www.zapimoveis.com.br/imovel/venda-cas...
4,"Rua Frutal, 203 - Bosque dos Eucaliptos, São J...",R$ 390.000,condomínio não informado,IPTU não informado,"[, , , , 120 m², 3 quartos, 2 vagas, 2 banheiros]","[Aceita animais, Varanda gourmet]",Esta casa está localizada no Bosque dos Eucali...,https://www.zapimoveis.com.br/imovel/venda-cas...
...,...,...,...,...,...,...,...,...
1195,"Residencial União, São José dos Campos - SP",R$ 426.000,condomínio não informado,IPTU não informado,"[, , , , 125 m², 2 quartos, 2 vagas, 2 banheiros]","[Área de serviço, Churrasqueira, Varanda]","REF 90230 - R$ R$ 426.000,00 - Maciel Negócios...",https://www.zapimoveis.com.br/imovel/venda-cas...
1196,"Jardim República, São José dos Campos - SP",R$ 426.000,condomínio não informado,IPTU R$ 500,"[, , , , 200 m², 5 quartos, 1 vaga, 5 banheiros]",[],Sobrado com 5 KITNETS 200 m² - Jardim Repúblic...,https://www.zapimoveis.com.br/imovel/venda-cas...
1197,"Vila São Pedro, São José dos Campos - SP",R$ 426.000,condomínio não informado,IPTU não informado,"[, , , , 300 m², 3 quartos, 3 vagas, 2 banheiros]",[],3 dormitórios\nSala\nCopa\nCozinha\n2 banheiro...,https://www.zapimoveis.com.br/imovel/venda-cas...
1198,"Primavera 1B, São José dos Campos - SP",R$ 426.000,condomínio não informado,IPTU não informado,"[, , , 120 m², 2 quartos, 1 banheiro]",[],Vendo casa no Jardim Primavera São José dos Ca...,https://www.zapimoveis.com.br/imovel/venda-cas...


In [23]:
data_atual = date.today()
data_atual = f'{data_atual.day}_{data_atual.month}_{data_atual.year}'
tabela_imoveis_raw = dados2.to_csv(f'tabela_imoveis_new2_raw_{data_atual}.csv', sep = ',', index = False)


In [25]:
df = pd.read_csv('tabela_imoveis_new_raw_13_3_2022.csv')
df

,Endereço,Preço,Condominio,Iptu,Lista Comodos,Lista Caracteristicas,Obserevacoes,url_id_anuncio
0,"Rua Ibiuna, 35 - Bosque dos Eucaliptos, São Jo...",R$ 350.000,condomínio não informado,IPTU não informado,"['', '', '', '148 m²', '3 quartos', '2 banheir...",[],Sobrado\nSol da manhã\nSala para 2 ambientes\n...,https://www.zapimoveis.com.br/imovel/venda-cas...
1,"Rua Santo Expedito - Residencial União, São Jo...",R$ 350.000,condomínio não informado,IPTU não informado,"['', '', '', '148 m²', '3 quartos', '2 banheir...","['Área de serviço', 'Cozinha', 'Quintal']","Sobrado no residencial União, em São José dos ...",https://www.zapimoveis.com.br/imovel/venda-cas...
2,"Jardim Limoeiro, São José dos Campos - SP",R$ 350.000,condomínio não informado,IPTU não informado,"['', '', '', '', '100 m²', '3 quartos', '2 vag...",['Circuito de segurança'],(28/02) - Ref. Imovel: V46136SA - Casa Padrao ...,https://www.zapimoveis.com.br/imovel/venda-cas...
3,"Vila Tatetuba, São José dos Campos - SP",R$ 350.000,condomínio não informado,IPTU não informado,"['', '', '', '', '74 m²', '3 quartos', '2 vaga...","['Circuito de segurança', 'Elevador']",(28/02) - Ref. Imovel: V45462SA - Casa Padrao ...,https://www.zapimoveis.com.br/imovel/venda-cas...
4,"Vila das Flores, São José dos Campos - SP",R$ 350.000,condomínio não informado,IPTU R$ 400,"['', '', '', '', '179 m²', '3 quartos', '2 vag...",[],Sobrado no Vila Das Flores - Zona Sul de São J...,https://www.zapimoveis.com.br/imovel/venda-cas...
...,...,...,...,...,...,...,...,...
1195,"Rua Frutal, 203 - Bosque dos Eucaliptos, São J...",R$ 390.000,condomínio não informado,IPTU R$ 50,"['', '', '', '', '120 m²', '3 quartos', '2 vag...",[],ANALISAMOS TODAS AS PROPOSTAS\nA casa no bairr...,https://www.zapimoveis.com.br/imovel/venda-cas...
1196,"Rua Wilma Vinhas Santos, 203 - Jardim Portugal...",R$ 390.000,condomínio não informado,IPTU R$ 256,"['', '', '', '', '125 m²', '2 quartos', '2 vag...","['Aceita animais', 'Closet', 'Mobiliado']",Casa com ótimo estado de conservação. Bairro r...,https://www.zapimoveis.com.br/imovel/venda-cas...
1197,"Vila das Flores, São José dos Campos - SP",R$ 390.000,condomínio não informado,IPTU não informado,"['', '', '', '', '75 m²', '3 quartos', '2 vaga...","['Área de serviço', 'Cozinha']",Vila das Flores que fica na Zona Sul de São Jo...,https://www.zapimoveis.com.br/imovel/venda-cas...
1198,"Vila das Flores, São José dos Campos - SP",R$ 390.000,condomínio não informado,IPTU não informado,"['', '', '', '', '90 m²', '3 quartos', '2 vaga...","['Aceita animais', 'Varanda gourmet']",Casa térrea novinha não foi habitada ainda no ...,https://www.zapimoveis.com.br/imovel/venda-cas...
